In [11]:
# HA MINH HIEU 20220026


import os 
from Model.AMO import * 

file = ['gen0.txt', 'gen1.txt', 'gen2.txt', 'gen3.txt', 'gen4.txt']

# Parameters 
idx = 0
n_ants_idx = 4 
ALIMENTATION = True 
K_start = True 
Use_local_search = True 
n_ants = [10, 20, 50, 100, 200, 500]


idx = 0
n_ants_idx = 4 
ALIMENTATION = False 
K_start = True 
Use_local_search = True 
n_ants = [10, 20, 50, 100, 200, 500]


idx = 0
n_ants_idx = 4 
ALIMENTATION = True 
K_start = True 
Use_local_search = False 
n_ants = [10, 20, 50, 100, 200, 500]


idx = 0
n_ants_idx = 4 
ALIMENTATION = True 
K_start = False 
Use_local_search = False 
n_ants = [10, 20, 50, 100, 200, 500]




idx = 0
n_ants_idx = 1 
ALIMENTATION = True 
K_start = True 
Use_local_search = True 
n_ants = [10, 20, 50, 100, 200, 500]







with open('Data.txt', 'w') as outfile:
    file_path = 'data_gen/' + file[idx]
    with open(file_path) as infile:
        for line in infile:
            outfile.write(line)

model = Net3().to(device)
model.load_state_dict(torch.load('AMO-ACO-100.pt', map_location=torch.device('cpu')))


/var/folders/ht/lj5bk00s0t37_g3yjc_zb3zh0000gn/T/ipykernel_22601/509295110.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('AMO-ACO-100

<All keys matched successfully>

In [12]:
import torch
from torch.distributions import Categorical
import random
import itertools
import numpy as np
import copy
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy
import math 
import numpy as np
from Model.Config import *
from Model.AMO import * 
from Ant_k_starts import *
from Normalize_data import *
from Ant import *
from Injection import *
from Cross_Exchange import *
from Local_search import * 
from Draw import *


In [13]:
device = None
cfg = Data_100()
EPS = 1e-10
pyg_data_normalize = normalize_data(cfg)
heuristic_measure, log, topk = model(pyg_data_normalize)
heuristic_measure = heuristic_measure.reshape((cfg.graph_size+1, cfg.graph_size+1))
max_cap, xcoord, ycoord, demand, e_time, l_time, s_time, data = load_data()

data = torch.tensor([[float(x) for x in y] for y in data])
tsp_coordinates = data[:, 1:3] 
demands = torch.tensor(demand, dtype = torch.float32)
time_window = data[:, 4:]
durations = time_window[:, -1] 
distances = gen_distance_matrix(tsp_coordinates, device = device)

if K_start:
    aco = ACO(distances, demands, time_window, 10, topk, max_cap, heuristic=heuristic_measure, n_ants=n_ants[n_ants_idx])
else:
    aco = ACO(distances, demands, time_window, 10, topk, max_cap, heuristic=heuristic_measure, n_ants=n_ants[n_ants_idx], options=2)




In [14]:
max_cap, xcoord, ycoord, demand, e_time, l_time, s_time, data = load_data()

CAP=max_cap
colony=Ant(data,CAP,0.7, heuristic_measure)
colony.customer_cord()
colony.euclidean_distance()
colony.width_window()
_ = colony.path_pheromon()

In [15]:
def convert_dict(path): # path: tensor depot 0(m,)
    path += 1
    path = path.to(torch.long)
    zero_indices = torch.where(path == 1)[0]
    zero_indices = zero_indices.tolist()
    path = path.tolist()
    while zero_indices[-1] - zero_indices[-2] == 1:
        zero_indices.pop()
    dict = {}
    for i in range(len(zero_indices) - 1):
        dict[i] = path[zero_indices[i]: zero_indices[i+1] + 1 ]
    return dict

In [16]:
def update_btnt(path, cost, pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost, d = 2):
    '''
    path: Loi giai tiem nang
    cost: ham cost cua loi giai
    pheromone: aco.pheromone
    effort: so iter truoc khi ket thuc
    ''' 
    if cost < best_cost:
        reward = int(k_candidate * prob_to_update * reward_coff) * effort
    else:
        reward = int(k_candidate * prob_to_update) * effort
    if d == -1:
        reward = 1 
    for route in path.values(): # route
        for l in range(len(route) - 1):
            pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += reward/cost
    reward_coff += d
    return reward_coff, pheromone
    

AMO-ACO

In [17]:
import time
lstt = []
lstt1 = []
max_iteration = 200 
n_customer = len(data)
k_candidate = 100
prob_to_update = 0.05
best = 1e10
best_cost = 1e10
aco.decay = 1
aco.beta = 0.5
elitism_set = 0
reward_coff = 1 
effort = 0
final_path = 0
prob_to_destroy = 0.1
cnt = 0
counter = 0
best_data = []
best_cost_data = []
destroy_data = []
t1 = time.time()
def save_solution(ants_route, travel_distance, BTNT, max_route, max_travel):
    if len(ants_route.keys()) < max_route:
        BTNT[-1] = ants_route
        max_travel = travel_distance
        max_route = len(ants_route.keys())
    elif len(ants_route.keys()) == max_route and travel_distance < max_travel:
        BTNT[-1] = ants_route
        max_travel = travel_distance
        max_route = len(ants_route.keys())
    return BTNT, max_travel, max_route
BTNT = [0]
max_travel = 1e10
max_route = int(n_customer)
for k in range(max_iteration):
    paths, costs = aco.run()
    local_path = []
    candidate_values, indexs = torch.topk(costs, k = k_candidate, largest=False)
    candidate_path = paths.T[indexs] # (k * prob_size)
    for i, (value, path) in enumerate(zip(candidate_values, candidate_path)):
        ants_route = convert_dict(path)
        if torch.rand(1) < prob_to_update:
            travel_distance, ants_route = injection(ants_route, colony, 0.5)
            travel_distance, ants_route = cross_exchange(ants_route, colony)
            travel_distance, ants_route = ls_1route(ants_route, colony)
            travel_distance, ants_route = ls3(ants_route)
            BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
            candidate_values[i] = travel_distance
        local_path.append(ants_route)
    
    aco.pheromone *= aco.decay
    value_to_update, index_to_update = torch.topk(candidate_values, k = int(k_candidate * prob_to_update), largest=False)
    for i, j in enumerate(index_to_update):
        path = local_path[j] # dict
        for route in path.values(): # route
            for l in range(len(route) - 1):
                if i == 0:
                    aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
                aco.pheromone[int(route[l] - 1)][int(route[l+1] - 1)] += 1/value_to_update[i]
    if torch.min(candidate_values) < best:
        best = torch.min(candidate_values)
        best_path = local_path[index_to_update[0]]
        if k > 0 and elitism_set != 0 and ALIMENTATION:
            path, cost = elitism_set
            reward_coff, pheromone = update_btnt(path, cost, aco.pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost)
            aco.pheromone = pheromone 
        elitism_set = (best_path, best)  
        effort = 1
        tries = 0
    
    # Alimentation:
    if tries == 1 and elitism_set != 0 and ALIMENTATION: 
        tries = 0
        path, cost = elitism_set
        reward_coff, pheromone = update_btnt(path, cost, aco.pheromone, effort, k_candidate, prob_to_update, reward_coff, best_cost)
        elitism_set = 0
        aco.pheromone = pheromone 
    
    if elitism_set == 0:
        counter += 1
    
    if elitism_set != 0:
        counter = 0
        ants_route, cost = elitism_set
        for _ in range(1):
            travel_distance, ants_route = injection(ants_route, colony, 0.5)
        for _ in range(1):
            travel_distance, ants_route = cross_exchange(ants_route, colony)
        if Use_local_search:
            travel_distance, ants_route = local_search(ants_route, colony, n_customer, cfg.q)
            travel_distance, ants_route = ls3(ants_route)
        for _ in range(50):
            travel_distance, ants_route = ls_1route(ants_route, colony)
        BTNT, max_travel, max_route = save_solution(ants_route, travel_distance, BTNT, max_route, max_travel)
        if travel_distance < cost:
            elitism_set = (ants_route, travel_distance)
            effort += 1
            if travel_distance < best_cost:
                best_cost = travel_distance 
                final_path = ants_route
        else:
            effort += 1
            tries += 1
    
    if elitism_set == 0 and counter > prob_to_destroy * max_iteration and ALIMENTATION:
        print(destroy)
        travel_distance, ants_route = destroy(final_path, colony, 0.5)
        travel_distance, ants_route = destroy_ls_1route(final_path, colony)
        r = np.random.random()
        if r < 0.5:
            travel_distance, ants_route = ls1(final_path)
        else:
            travel_distance, ants_route = ls2(final_path)
        print("Destroy: ", travel_distance)
        elitism_set = (ants_route, travel_distance)
        counter = 0
        destroy_data.append(k)
    best_data.append(best)
    best_cost_data.append(best_cost)
    print('epoch {}: Best: {}, Alimentation: {}'.format(k, best, best_cost))
    if (k + 1) % 10 == 0:
        lstt.append((best_cost))
        lstt1.append((best)) 
t2 = time.time()
time_run = round(t2-t1, 2)



epoch 0: Best: 4229.54736328125, Alimentation: 4099.607953202592
epoch 1: Best: 4130.0498046875, Alimentation: 4077.609940940971
epoch 2: Best: 3913.304443359375, Alimentation: 3871.7665253953255
epoch 3: Best: 3913.304443359375, Alimentation: 3793.6733516000922
epoch 4: Best: 3913.304443359375, Alimentation: 3740.7253415773184
epoch 5: Best: 3913.304443359375, Alimentation: 3685.0536593706383
epoch 6: Best: 3913.304443359375, Alimentation: 3665.0529003298298
epoch 7: Best: 3913.304443359375, Alimentation: 3643.9486525926586
epoch 8: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 9: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 10: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 11: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 12: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 13: Best: 3913.304443359375, Alimentation: 3594.308585452165
epoch 14: Best: 3913.304443359375, Alimentation: 3594.308585452165
ep

KeyboardInterrupt: 

Optional

In [ ]:
with open('Data.txt', 'r') as f:
    name = f.readline()
name.rstrip()


with open('Solution.txt', 'a') as f:
    f.write(name + '\n')
    f.write('Time run: {}'.format(time_run)+ '\n')
    f.write('Best \n')
    for log in lstt1:
        f.write("{}".format(log)+ '-')
    f.write('\n')
    f.write('Alimentation \n')
    for log in lstt:
        f.write("{}".format(log)+ '-')
    f.write('\n')
    f.write("-----------------------------------------------"+ '\n')
